In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
import tensorflow as tf
import numpy as np
import time
import sys
import tensorflow_probability as tfp
import statistics

import torch

In [2]:
#Define a model as asked by the problem definition

def build_model(width, height, depth, classes):
	# initialize the input shape and channels dimension to be
	# "channels last" ordering
	inputShape = (height, width, depth)
	chanDim = -1
	# build the model using Keras' Sequential API
	model = Sequential([
		# CONV => RELU => POOL layer set
		Conv2D(30, (3, 3), padding="same", input_shape=inputShape),
		Activation("relu"),
		MaxPooling2D(pool_size=(2, 2)),
		# (CONV => RELU => BN) * 2 => POOL layer set
		Conv2D(50, (3, 3), padding="same"),
		Activation("relu"),
		# first (and only) set of FC => RELU layers
		Flatten(),
		Dense(200),
		Activation("relu"),
		# softmax classifier
		Dense(classes),
		Activation("softmax")
	])
	# return the built model to the calling function
	return model

In [3]:
# Returns gradient after taking a single step

def step(X, y, model):
  # keep track of our gradients
  with tf.GradientTape() as tape:
		# make a prediction using the model and then calculate the
		# loss
    pred = model(X)
    loss = categorical_crossentropy(y, pred)
	# calculate the gradients using our tape and then update the
	# model weights
  grads = tape.gradient(loss, model.trainable_variables)
  return(grads)
  # print(type(grads))
  # print(grads)
  # opt.apply_gradients(zip(grads, model.trainable_variables))

In [4]:
#----------------------------------- Testing median tensor creation ----------------------------------

# Leading the dataset

# initialize the number of epochs to train for, batch size, and
# initial learning rate
EPOCHS = 1
BS = 32
INIT_LR = 0.005
# load the MNIST dataset
print("[INFO] loading MNIST dataset...")
((trainX, trainY), (testX, testY)) = mnist.load_data()
# add a channel dimension to every image in the dataset, then scale
# the pixel intensities to the range [0, 1]
trainX = np.expand_dims(trainX, axis=-1)
testX = np.expand_dims(testX, axis=-1)
trainX = trainX.astype("float32") / 255.0
testX = testX.astype("float32") / 255.0
# one-hot encode the labels
trainY = to_categorical(trainY, 10)
testY = to_categorical(testY, 10)

[INFO] loading MNIST dataset...


In [5]:
# Spliting the dataset among 10 machines

num_machines = 10
num_train = int(trainX.shape[0])
num_train_per_machine = int(num_train/num_machines)

#Split training and test data among 10 machines
for i in range(num_machines):
  globals()['X_train_%s' % str(i+1)] = trainX[i*num_train_per_machine: (i+1)*num_train_per_machine]
  globals()['y_train_%s' % str(i+1)] = trainY[i*num_train_per_machine: (i+1)*num_train_per_machine]

In [6]:
# build our model and initialize our optimizer
print("[INFO] creating model...")
model_median = build_model(28, 28, 1, 10)
opt = SGD(learning_rate=INIT_LR)

[INFO] creating model...


In [7]:
# in order to calculate accuracy using Keras' functions we first need
# to compile the model
model_median.compile(optimizer=opt, loss=categorical_crossentropy,
	metrics=["acc"])
# now that the model is compiled we can compute the accuracy
(loss, acc) = model_median.evaluate(testX, testY)
print("[INFO] test accuracy: {:.4f}".format(acc))

313/313 [==============================] - 5s 14ms/step - loss: 2.3138 - acc: 0.0617
[INFO] test accuracy: 0.0617


In [8]:
label_switch_attack_machines = [3]

loss_list = []
acc_list = []

for i in range(1000):
  print("------------------------"+str(i)+"-----------------------------")
  if(i%10 == 0):
    (loss, acc) = model_median.evaluate(testX, testY)
    print("[INFO] test accuracy: {:.4f}".format(acc))
    loss_list.append((i,loss))
    acc_list.append((i,acc))

  gradient_median_array = []
  
  gradient_shapes = []
  # gradient_median_tensor_store = []
  # gradient_median_tensor_flatten_store = []
  
  #Defining 8 arrays to hold flattened grads of each machine - Size = 1x10
  for temp in range(8):
    globals()['grads_flattened_%s' % str(temp+1)] = None

  #For each machine, select a batch of 32 training data and perform one feedforward and get gradients
  for machine in range(10):
    index = np.random.choice(globals()['X_train_%s' % str(machine+1)].shape[0], 32, replace=False)
    train_X_batch = globals()['X_train_%s' % str(machine+1)][index].copy()
    #train_X_batch = train_X_batch.reshape(32,28,28,1)
    train_y_batch = globals()['y_train_%s' % str(machine+1)][index].copy()

    #Label switch attack
    if(machine in label_switch_attack_machines):
      print(machine)
      for batch_element in range(32):
        for pos in range(10):
          if(train_y_batch[batch_element][pos]==1):
            train_y_batch[batch_element][pos]=0.
            train_y_batch[batch_element][9-pos]=1.

    globals()['grads_median_%s' % str(machine+1)] = step(train_X_batch, train_y_batch, model_median)
    #print("######### Tensor shape 1 #############")
    #print(globals()['grads_median_%s' % str(machine+1)][0])
    #original_shape = globals()['grads_median_%s' % str(machine+1)][0].shape
    #tensor_flatten = tf.reshape(globals()['grads_median_%s' % str(machine+1)][0], [-1])
    #print(tensor_flatten)
    #tensor_original = tf.reshape(tensor_flatten, original_shape)
    #print(tensor_original)
    #print(tf.equal(globals()['grads_median_%s' % str(machine+1)][0], tensor_original))

    #print(globals()['grads_median_%s' % str(machine+1)][0])
    #loop through the elements (8) of the grads and store them in a array
    for grads in range(len(globals()['grads_median_%s' % str(machine+1)])):
      if(globals()['grads_flattened_%s' % str(grads+1)] == None):
        #globals()['grads_flattened_%s' % str(grads+1)] = [tf.reshape(globals()['grads_median_%s' % str(machine+1)][grads], [-1])]
        torch_tensor = torch.from_numpy(tf.reshape(globals()['grads_median_%s' % str(machine+1)][grads], [-1]).numpy())
        torch_tensor = torch_tensor.unsqueeze(0)
        #print(torch_tensor.size())
        globals()['grads_flattened_%s' % str(grads+1)] = torch_tensor
        gradient_shapes.append(globals()['grads_median_%s' % str(machine+1)][grads].shape)
      else:
        torch_tensor = torch.from_numpy(tf.reshape(globals()['grads_median_%s' % str(machine+1)][grads], [-1]).numpy())
        torch_tensor = torch_tensor.unsqueeze(0)
        #print(torch_tensor.size())
        globals()['grads_flattened_%s' % str(grads+1)] = torch.cat([globals()['grads_flattened_%s' % str(grads+1)], torch_tensor], 0)
        gradient_shapes.append(globals()['grads_median_%s' % str(machine+1)][grads].shape)

  for temp in range(8):
      #print(globals()['grads_flattened_%s' % str(temp+1)].size())
      gradient_median = torch.median(globals()['grads_flattened_%s' % str(temp+1)], 0)
      #print(gradient_median.values.size())
      gradient_median_numpy = gradient_median.values.numpy()
      tf_tensor = tf.convert_to_tensor(gradient_median_numpy)
      tf_tensor = tf.reshape(tf_tensor, gradient_shapes[temp])
      #print(tf_tensor)
      gradient_median_array.append(tf_tensor)

  #print(gradient_median_array[0])

  # final_grad_tensor = []
  # for grad_run in range(8):
  #   grad_tensor = []
  #   print(globals()['grads_flattened_%s' % str(grad_run+1)][0].shape[0])
  #   for dim_run in range(globals()['grads_flattened_%s' % str(grad_run+1)][0].shape[0]):
  #     arr=[]
  #     for machine_run in range(num_machines):
  #       arr.append(globals()['grads_flattened_%s' % str(grad_run+1)][machine_run][dim_run])
  #     grad_tensor.append(statistics.median(arr))
  #     del[arr[:]]
  #   grad_tensor = tf.convert_to_tensor(grad_tensor, dtype=tf.float32)
  #   final_grad_tensor.append(tf.reshape(grad_tensor, gradient_shapes[grad_run]))
    #del(grad_tensor[:])

  # Calculate median, trimmed mean for gradient array from all machines and apply to model
  #opt.apply_gradients(zip(gradient_mean, model_mean.trainable_variables))
  opt.apply_gradients(zip(gradient_median_array, model_median.trainable_variables))
  #del(final_grad_tensor[:])
  #opt.apply_gradients(zip(gradient_trimmed_mean, model_trimmed_mean.trainable_variables))



# print("------------------Mean------------------")
# print(gradient_mean)
	# show timing information for the epoch

------------------------0-----------------------------
313/313 [==============================] - 5s 14ms/step - loss: 2.3138 - acc: 0.0617
[INFO] test accuracy: 0.0617
3
------------------------1-----------------------------
3
------------------------2-----------------------------
3
------------------------3-----------------------------
3
------------------------4-----------------------------
3
------------------------5-----------------------------
3
------------------------6-----------------------------
3
------------------------7-----------------------------
3
------------------------8-----------------------------
3
------------------------9-----------------------------
3
------------------------10-----------------------------
313/313 [==============================] - 5s 15ms/step - loss: 1.7986 - acc: 0.6390
[INFO] test accuracy: 0.6390
3
------------------------11-----------------------------
3
------------------------12-----------------------------
3
------------------------13--

In [9]:
# in order to calculate accuracy using Keras' functions we first need
# to compile the model
# model_median.compile(optimizer=opt, loss=categorical_crossentropy,
# 	metrics=["acc"])
# now that the model is compiled we can compute the accuracy
(loss, acc) = model_median.evaluate(testX, testY)
print("[INFO] test accuracy: {:.4f}".format(acc))

f = open("loss_median_attack.txt", "w")
for element in loss_list:
    f.write(str(element) + "\n")
f.close()

f = open("acc_median_attack.txt", "w")
for element in acc_list:
    f.write(str(element) + "\n")
f.close()

313/313 [==============================] - 5s 15ms/step - loss: 2.3021 - acc: 0.1010
[INFO] test accuracy: 0.1010


In [10]:
a = torch.randn(10,5)
print(a)
b = torch.median(a, 0)
print(b)

tensor([[-0.3793,  0.4960, -1.6193,  0.5484,  0.0593],
        [ 2.4112, -0.2042, -0.3294,  0.3131,  0.4647],
        [-0.4781,  1.8824, -1.0203,  0.0575,  2.0246],
        [ 1.0997,  0.1096,  0.8209, -1.4449, -1.0764],
        [ 0.7642,  0.2134, -0.3670, -0.3569,  0.0062],
        [ 0.0391, -1.1315, -0.3807, -1.6065,  0.6501],
        [-0.8608, -1.7519,  0.3694,  0.2111,  0.6928],
        [ 1.1051, -0.4345,  0.5809, -0.2814,  0.3232],
        [-0.3423,  0.2823,  0.0476, -0.6518,  0.0052],
        [-1.0644,  0.2154, -0.9499, -2.1006, -0.8843]])
torch.return_types.median(
values=tensor([-0.3423,  0.1096, -0.3670, -0.3569,  0.0593]),
indices=tensor([8, 3, 4, 4, 0]))


In [11]:
A = torch.randn(1,10)
B = torch.randn(1,10)
print(A)
print(B)
C = torch.cat((A, B), 0)
D = torch.cat((C, A), 0)
print(D)
#torch.stack([A, B], dim=0)

tensor([[ 0.4484,  0.1994,  1.0324, -1.1988, -1.1225, -1.1755,  0.3265,  0.7158,
         -1.2649, -0.8127]])
tensor([[ 0.3088, -1.4703, -1.9804, -0.4706,  0.5769,  1.1226, -0.6026,  1.4699,
         -0.1097, -0.4823]])
tensor([[ 0.4484,  0.1994,  1.0324, -1.1988, -1.1225, -1.1755,  0.3265,  0.7158,
         -1.2649, -0.8127],
        [ 0.3088, -1.4703, -1.9804, -0.4706,  0.5769,  1.1226, -0.6026,  1.4699,
         -0.1097, -0.4823],
        [ 0.4484,  0.1994,  1.0324, -1.1988, -1.1225, -1.1755,  0.3265,  0.7158,
         -1.2649, -0.8127]])
